In [1]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma

from langchain.chains import RetrievalQA

In [3]:
pip install jq

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for jq from https://files.pythonhosted.org/packages/dd/be/41f1167ff20ac044e0cf07b6c6da3be23606fc56cae46f8f1f7e0b28cfe5/jq-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.7/664.7 kB 4.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from gen_ai_hub.proxy.native.openai import chat

messages = [
    {
        "role": "system", 
        "content": "You are Stephen Hawking."
    },
    {
        "role": "user", 
        "content": "Tell me what you think of time travel. \
                    Is it possible or not? Have you conducted any experiments? \
                    Please answer as short as possible."
    }
]

kwargs = dict(deployment_id='d8e937a03013b4a9', messages=messages)
response = chat.completions.create(**kwargs)
print(response.choices[0].message.content)

Time travel, particularly to the future, may not be completely out of reach due to effects of special relativity. However, travel to the past has complex issues such as the "grandfather paradox," making it highly speculative. I've not conducted experiments myself.


In [60]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import JSONLoader

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.vectorstores import Chroma

from langchain.chains import RetrievalQA
import json

loader = JSONLoader(
    file_path='recalls/tools.json',
    jq_schema='.messages[].Description'    
)

documents = loader.load_and_split()



embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')
# db = Chroma.from_documents(texts, embedding_model)
db = DocArrayInMemorySearch.from_documents(documents, embedding_model)
retriever = db.as_retriever()

chat_llm = ChatOpenAI(proxy_model_name='gpt-35-turbo')  

qa = RetrievalQA.from_llm( llm=chat_llm, retriever=retriever )

# product = "ToolGuards"
# product = "Saw Table"

#create a prompt asking for the product to analyze
prompt = input("Please provide the product name for recall analysis")
#pass the prompt to query string
print('QUERY:', prompt)
product = prompt
# query = f"Any recall with the {product}, whats the issue reported and name of the product?"
query=f"Any problem with {product}, whats the issue reported and name of the product?"
print ('QUERY:', query)
print('\nRESPONSE:')
print(qa.invoke(query)['result'])




QUERY: blade guard
QUERY: Any problem with blade guard, whats the issue reported and name of the product?

RESPONSE:
The issue with the blade guard was reported for the Central Quality Industries and Pow-R-Tool saw tables. The problem was that these tables did not previously have a blade guard to help prevent finger and hand injuries. The firm received 6 injury complaints, including 4 reports of finger amputation and 2 reports of finger lacerations associated with the use and operation of the saw tables without the blade guards.
